This script is used for:
- adjusting the latitude and longtitude of the urban surface data     
- converting urban surface data from xarray to dataframe and save as a parquet file   

How to launch it:   
```bash
execcasper -A your_project -l gpu_type=v100 -l walltime=06:00:00 -l select=1:ncpus=18:mpiprocs=36:ngpus=1:mem=100GB
bash aws_urban_env.sh
```

In [1]:
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cftime

In [2]:
urban_LE_path = "/glade/scratch/zhonghua/urban_params/urban_LE/"
urban_surf_path = "/glade/scratch/zhonghua/urban_params/urban_surface.nc"

fd = {
    "label":"TREFMXAV_U",
    "CAM": ['FLNS','FSNS','PRECT','PRSN','QBOT','TREFHT','UBOT','VBOT'],
    "surf":['CANYON_HWR','EM_IMPROAD','EM_PERROAD','EM_ROOF','EM_WALL', 
            'HT_ROOF','THICK_ROOF','THICK_WALL','T_BUILDING_MAX','T_BUILDING_MIN',
            'WTLUNIT_ROOF','WTROAD_PERV','NLEV_IMPROAD','PCT_URBAN',
            'ALB_IMPROAD','ALB_PERROAD','ALB_ROOF','ALB_WALL',
            'TK_ROOF','TK_WALL','CV_ROOF','CV_WALL',
            'TK_IMPROAD_0','CV_IMPROAD_0','TK_IMPROAD_1','CV_IMPROAD_1'],
    "loc":["lat","lon"]
}

def load_urban_data(member, start_year, end_year, urban_LE_path=urban_LE_path, urban_surf_path=urban_surf_path):
    # load urban climate Xarray Dataset, convert the fortmat of time, and load urban surface Xarray Dataset
    ds_urban_LE = xr.open_dataset(urban_LE_path+member+"_"+start_year+"_"+end_year+".nc")
    ds_urban_LE = ds_urban_LE.assign_coords(time = ds_urban_LE.indexes['time'].to_datetimeindex())
    # rename the coordinate
    ds_urban_surf = xr.open_dataset(urban_surf_path)\
                    .rename_dims({"lsmlat":"lat", "lsmlon":"lon"})\
                    .assign_coords(lat = ds_urban_LE.lat, lon = ds_urban_LE.lon)
    return ds_urban_surf.to_dataframe()

## save urban surface only

In [3]:
%%time
train = load_urban_data("003", "2006", "2015")
print(train.shape)

(55296, 26)
CPU times: user 213 ms, sys: 209 ms, total: 422 ms
Wall time: 697 ms


In [4]:
%%time
train.to_parquet("/glade/scratch/zhonghua/urban_params/urban_surface.parquet.gzip",
                compression="gzip", engine="fastparquet")
del train
gc.collect()

CPU times: user 234 ms, sys: 9.67 ms, total: 244 ms
Wall time: 332 ms


425